In [1266]:
%%capture
%pip install qiskit qiskit-aer pandas pylatexenc matplotlib

In [1267]:
from qiskit import __version__
__version__

'2.2.1'

In [1268]:
import numpy as np
import pandas as pd

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aer import AerSimulator

simulator = AerSimulator()

## Quantum doors

X door -> Invert

H door -> Hadamard (Sobreposition)

SWAP door -> Send to another channel

Take a look at https://quantum.cloud.ibm.com/composer?initial=N4IgjghgzgtiBcIDyAFAogOQIoEEDKAsgAQBMAdAAwDcAOgHYCWdAxgDYCuAJgKZE3jdWDAEYBGMk2b9ademABO3AOZEwAbQAsAXRnNFK5pp316IADQg6EGNwQgAqnQAuDJ626cizBvObtXIAC+QA

In [1269]:
A = QuantumRegister(1, 'A')
E = QuantumRegister(1, 'E')
B = QuantumRegister(1, 'B')

MB = ClassicalRegister(1, 'MB')
ME = ClassicalRegister(1, 'ME')

## Without Eve

In [1270]:
situation = {}
situation['alice_bit'] = np.random.choice([0, 1])
situation['alice_h'] = np.random.choice([True, False])
situation['bob_h'] = np.random.choice([True, False])
situation

{'alice_bit': np.int64(1), 'alice_h': np.False_, 'bob_h': np.True_}

In [1271]:
qc = QuantumCircuit(A, E, B, MB, ME)

# We're going to use a few more commands here:
# qc.h
# qc.x
# qc.swap
# qc.barrier()
# qc.measure(q, E)

## === Alice ===
if situation['alice_bit'] == 1:
    qc.x(A)
if situation['alice_h']:
    qc.h(A)
qc.swap(A,E)

## === Eve ===
qc.barrier()
qc.swap(E, B)

## === Bob ===
qc.barrier()
if situation['bob_h']:
    qc.h(B)
qc.measure(B,MB)


qc.draw()

┌───┐    ░     ░         
   A: ┤ X ├─X──░─────░─────────
      └───┘ │  ░     ░         
   E: ──────X──░──X──░─────────
               ░  │  ░ ┌───┐┌─┐
   B: ─────────░──X──░─┤ H ├┤M├
               ░     ░ └───┘└╥┘
MB: 1/═══════════════════════╩═
                             0 
ME: 1/═════════════════════════

Let's simulate a communication without Eve

In [1272]:
job = simulator.run(qc, shots=100)
counts = job.result().get_counts()
# counts = job.result().results[0].data.counts
counts

{'0 0': 49, '0 1': 51}

In [1273]:
situation['bob_bit'] = 0 if '0 0' in counts else 1
# situation['bob_bit'] = 0 if '0x0' in counts else 1
situation

{'alice_bit': np.int64(1),
 'alice_h': np.False_,
 'bob_h': np.True_,
 'bob_bit': 0}

In summary

In [1274]:
data = []

for _ in range(30):
    situation = {}
    situation['alice_bit'] = np.random.choice([0, 1])
    situation['alice_h'] = np.random.choice([True, False])
    situation['bob_h'] = np.random.choice([True, False])

    qc = QuantumCircuit(A, E, B, MB, ME)

    ## === Alice ===
    if situation['alice_bit'] == 1:
        qc.x(A)
    if situation['alice_h']:
        qc.h(A)
    qc.swap(A,E)

    ## === Eve ===
    qc.barrier()
    qc.swap(E, B)

    ## === Bob ===
    qc.barrier()
    if situation['bob_h']:
        qc.h(B)
    qc.measure(B,MB)

    job = simulator.run(qc, shots=100)
    counts = job.result().get_counts()
    # counts = job.result().results[0].data.counts

    situation['bob_bit'] = 0 if '0 0' in counts else 1
    # situation['bob_bit'] = 0 if '0x0' in counts else 1

    data.append(situation)

In [1275]:
df = pd.DataFrame(data)
df['match'] = df.apply(lambda row: '✅' if row['alice_h'] == row['bob_h'] else '❌', axis=1)
df

,alice_bit,alice_h,bob_h,bob_bit,match
0,0,False,False,0,✅
1,0,False,False,0,✅
2,1,True,False,0,❌
3,0,True,True,0,✅
4,0,True,True,0,✅
5,1,False,False,1,✅
6,0,True,False,0,❌
7,1,True,False,0,❌
8,1,True,True,1,✅
9,1,False,False,1,✅


In [1276]:
df = df[df['alice_h'] == df['bob_h']]
df

,alice_bit,alice_h,bob_h,bob_bit,match
0,0,False,False,0,✅
1,0,False,False,0,✅
3,0,True,True,0,✅
4,0,True,True,0,✅
5,1,False,False,1,✅
8,1,True,True,1,✅
9,1,False,False,1,✅
10,0,True,True,0,✅
13,0,False,False,0,✅
15,0,False,False,0,✅


## With Eve

In [1277]:
data = []

for _ in range(30):
    situation = {}
    situation['alice_bit'] = np.random.choice([0, 1])
    situation['alice_h'] = np.random.choice([True, False])
    situation['bob_h'] = np.random.choice([True, False])
    
    situation['eve_h_pre'] = np.random.choice([True, False])
    situation['eve_h_post'] = np.random.choice([True, False])

    qc = QuantumCircuit(A, E, B, MB, ME)

    ## === Alice ===
    if situation['alice_bit'] == 1:
        qc.x(A)
    if situation['alice_h']:
        qc.h(A)
    qc.swap(A,E)

    ## === Eve ===
    qc.barrier()
    if situation['eve_h_pre']:
        qc.h(E)
    qc.measure(E, ME)
    if situation['eve_h_post']:
        qc.h(E)
    qc.swap(E, B)

    ## === Bob ===
    qc.barrier()
    if situation['bob_h']:
        qc.h(B)
    qc.measure(B,MB)

    job = simulator.run(qc, shots=100)
    counts = job.result().get_counts()

    situation['bob_bit'] = 0 if ('0 0' in counts or '1 0' in counts) else 1
    
    situation['eve_bit'] = 0 if ('0 0' in counts or '0 1' in counts) else 1

    data.append(situation)
    
qc.draw()

┌───┐    ░             ░    
   A: ┤ H ├─X──░─────────────░────
      └───┘ │  ░ ┌─┐┌───┐    ░    
   E: ──────X──░─┤M├┤ H ├─X──░────
               ░ └╥┘└───┘ │  ░ ┌─┐
   B: ─────────░──╫───────X──░─┤M├
               ░  ║          ░ └╥┘
MB: 1/════════════╬═════════════╩═
                  ║             0 
ME: 1/════════════╩═══════════════
                  0

In [1278]:
counts

{'1 1': 25, '0 1': 28, '1 0': 24, '0 0': 23}

Remember that the results are presented from left to right, down to up

'1 1' -> ME = 1 and MB = 1

'0 1' -> ME = 0 and MB = 1

'0 0' -> ME = 0 and MB = 0

'1 0' -> ME = 1 and MB = 0

In [1280]:
df = pd.DataFrame(data)
df['match'] = df.apply(lambda row: '✅' if row['alice_h'] == row['bob_h'] else '❌', axis=1)
df['detected'] = df.apply(lambda row: '⚠️' if ((row['alice_h'] != row['bob_h']) and (row['alice_h'] == row['eve_bit'])) else '✔️', axis=1)
df['undetected'] = df.apply(lambda row: '☠️' if ((row['alice_h'] == row['bob_h']) and (row['alice_h'] == row['eve_bit'] == row['bob_bit'])) else '✔️', axis=1)
df

,alice_bit,alice_h,bob_h,eve_h_pre,eve_h_post,bob_bit,eve_bit,match,detected,undetected
0,1,False,False,True,True,0,0,✅,✔️,☠️
1,0,False,True,True,False,0,0,❌,⚠️,✔️
2,0,False,False,False,True,0,0,✅,✔️,☠️
3,1,True,True,False,False,0,0,✅,✔️,✔️
4,1,False,False,False,False,1,1,✅,✔️,✔️
5,0,False,True,True,False,0,0,❌,⚠️,✔️
6,1,False,True,False,True,1,1,❌,✔️,✔️
7,1,False,False,False,False,1,1,✅,✔️,✔️
8,0,False,True,False,False,0,0,❌,⚠️,✔️
9,0,True,True,True,True,0,0,✅,✔️,✔️


In [ ]:
df = df[df['alice_h'] == df['bob_h']]
df